In [9]:
__author__ = "rravishankar"

import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.grid_search import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

def warn(*args, **kwargs): pass
import warnings
warnings.warn = warn

In [2]:
#Training data 
train_data = pd.read_csv('../train.csv')
x_train = train_data.drop(['id','species'], axis = 1).values

#Encoding the names of plant species
label_encoder = LabelEncoder().fit(train_data['species'])
y_train = label_encoder.transform(train_data['species'])

In [3]:
#Standardization of the features
scaler = StandardScaler().fit(x_train)
x_train = scaler.transform(x_train)

In [4]:
#Building the Logistic Regression classifier
params = {'C' : [100, 1000],'tol' : [0.001, 0.0001]}
logistic_regression = LogisticRegression(solver = 'lbfgs', multi_class = 'multinomial')
clf = GridSearchCV(logistic_regression, params, scoring = 'log_loss', cv = 5)
clf.fit(x_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': [100, 1000], 'tol': [0.001, 0.0001]},
       pre_dispatch='2*n_jobs', refit=True, scoring='log_loss', verbose=0)

In [5]:
print(clf.best_params_)
print(clf.grid_scores_)

{'C': 1000, 'tol': 0.001}
[mean: -0.04409, std: 0.02003, params: {'C': 100, 'tol': 0.001}, mean: -0.04416, std: 0.02001, params: {'C': 100, 'tol': 0.0001}, mean: -0.04090, std: 0.02316, params: {'C': 1000, 'tol': 0.001}, mean: -0.04130, std: 0.02419, params: {'C': 1000, 'tol': 0.0001}]


In [6]:
#Test data
test_data = pd.read_csv('../test.csv')
test_id = test_data.pop('id')
x_test = test_data.values
x_test = scaler.transform(x_test)

In [7]:
#Predicting the probabiity of each category
y_test = clf.predict_proba(x_test)

In [8]:
#Writing the predictions to csv file
submission = pd.DataFrame(y_test, index = test_id, columns = label_encoder.classes_)
submission.to_csv('submissions.csv')